# Solar Panel Customer Support - Setup

In this notebook, we'll set up Amazon resources for our Solar Panel Customer Support Agent. We'll be:

1. Creating [Amazon Bedrock Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/) for solar panel installation and maintenance information from text files.
2. Creating [Amazon Bedrock Guardrails](https://aws.amazon.com/bedrock/guardrails/) to safeguard customer interactions.
3. Creating [Amazon DynamoDb](https://aws.amazon.com/dynamodb/) for storing customer profiles and data.
 

These components will later be integrated into our Solar Panel Customer Support agent.

##  Import Required Libraries

In [ ]:
# Install prerequisites
!pip install --upgrade -q boto3
%pip install -qr requirements.txt

In [ ]:
import os
import time
import boto3
import logging
import pprint
import json
import uuid
import requests

# Set up logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Set up AWS Clients
Please set up your aws credentials in your enviroment.

In [ ]:
# Set up AWS clients
session = boto3.Session()
region = session.region_name
sts_client = session.client('sts')
s3_client = session.client('s3')
bedrock_client = session.client('bedrock')
account_id = sts_client.get_caller_identity()["Account"]
account_id_suffix = [:3]
#suffix = f"{region}-{account_id_suffix}"
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
bedrock_runtime = boto3.client('bedrock-runtime')

# Display region and account information
print(f"Region: {region}")
#print(f"Account ID: {account_id}")

In [ ]:
# Get the current timestamp
current_time = time.time()
# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

## Amazon bedrock Knowledge Base for Solar Panel Manuals

Download Amazon Bedrock Knowledge Bases helper

In [ ]:
url = "https://raw.githubusercontent.com/aws-samples/amazon-bedrock-samples/main/rag/knowledge-bases/features-examples/utils/knowledge_base.py"
target_path = "utils/knowledge_base.py"
response = requests.get(url)
with open(target_path, "w") as f:
    f.write(response.text)
print(f"Downloaded Knowledge Bases utils to {target_path}")

## Create Amazon Bedrock Knowledge Base
In this section we will configure the Amazon Bedrock Knowledge Base containing the solar panel manuals for installation and maintainence. We will be using Amazon Opensearch Serverless Service as the underlying vector store and Amazon S3 as the data source containing the files.

In [ ]:
from utils.knowledge_base import BedrockKnowledgeBase

knowledge_base_name = f"solar-panel-manuals-knowledge-base-{suffix}"
knowledge_base_description = "Solar Panels Customer support Manuals."
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

For this notebook, we'll create a Knowledge Base with an Amazon S3 data source.

In [ ]:
data_bucket_name = f'solar-panel-support-agent-{suffix}-bucket' # replace it with your first bucket name.
data_sources=[{"type": "S3", "bucket_name": data_bucket_name}]

### Create the Amazon S3 bucket and upload the sample documents

In [ ]:
import botocore
import os

def create_s3_bucket(bucket_name, region=None):
    s3 = boto3.client('s3', region_name=region)

    try:
        if region is None or region == 'us-east-1':
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"✅ Bucket '{bucket_name}' created successfully.")
    except botocore.exceptions.ClientError as e:
        print(f"❌ Failed to create bucket: {e.response['Error']['Message']}")

create_s3_bucket(data_bucket_name, region)


In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

In [ ]:
upload_directory("./data", data_bucket_name)

### Create the Knowledge Base
We are now going to create the Knowledge Base using the abstraction located in the helper function we previously downloaded.

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

### Start ingestion job
Once the KB and data source created, we can start the ingestion job for the data source. During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()

### Test the Knowledge Base
We can now test the Knowledge Base to verify the documents have been ingested properly.

In [ ]:
query = "How to install sunpower X?"
foundation_model = "anthropic.claude-3-5-sonnet-20240620-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

In [ ]:
os.environ['AWS_REGION'] = region
os.environ['KNOWLEDGE_BASE_ID'] = kb_id
os.environ['MIN_SCORE'] = "0.1"

##  Create Amazon Bedrock Guardrail

Now we'll create a guardrail for our Solar Panel Customer Support agent to ensure safe and appropriate responses.

In [ ]:
guardrail_name=f"customer-support-{suffix}"
guardrail_description="This guardrail is for a Solar Panel Customer support agent to avoid unsafe, violations or investment conversations"

In [ ]:
try:
    # Check if guardrail already exists
    guardrail_exists = False
    existing_guardrail_id = None
    
    response = bedrock_client.list_guardrails(maxResults=100)
    for guardrail in response.get('guardrails', []):
        if guardrail.get('name') == guardrail_name:
            guardrail_exists = True
            existing_guardrail_id = guardrail.get("id")
            break
    
    # If guardrail exists, delete it to recreate
    if guardrail_exists:
        print(f"Guardrail {guardrail_name} already exists, deleting it to recreate...")
        bedrock_client.delete_guardrail(guardrailIdentifier=existing_guardrail_id)
        time.sleep(30)  # Wait for deletion to complete
    
    # Create new guardrail
    print(f"Creating guardrail {guardrail_name}...")
    response = bedrock_client.create_guardrail(
    name='customer-support-guardrail',
    description='Prevents the model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {
                'type': 'PROFANITY'
            }
        ]
    },
    blockedInputMessaging='I apologize, but I am not able to provide an answer to that question.',
    blockedOutputsMessaging='I apologize, but I am not able to provide an answer to that question.',
)
    
    guardrail_id = response['guardrailId']
    guardrail_version = response['version']
    
    print(f"Successfully created guardrail with ID {guardrail_id} and version {guardrail_version}")
    
except Exception as e:
    print(f"Error creating guardrail: {e}")

## Test the Guardrail

Let's test our guardrail to ensure it's working properly.

In [ ]:
# Test function to check if input/output is blocked by guardrail
def test_guardrail(text, source_type='INPUT'):
      response = bedrock_runtime.apply_guardrail(
          guardrailIdentifier=guardrail_id,
          guardrailVersion=guardrail_version,
          source=source_type,  # can be 'INPUT' or 'OUTPUT'
          content=[{"text": {"text": text}}]
      )

      # New response format uses different fields
      print(f"Action: {response.get('action')}")
      print(f"Action Reason: {response.get('actionReason', 'None')}")

      # Check if content was blocked
      is_blocked = response.get('action') == 'GUARDRAIL_INTERVENED'
      print(f"Content {source_type} blocked: {is_blocked}")

      if is_blocked:
          # Print topic policies that were triggered
          assessments = response.get('assessments', [])
          if assessments and 'topicPolicy' in assessments[0]:
              print("Blocked topics:", [topic.get('name') for topic in
  assessments[0]['topicPolicy'].get('topics', [])
                                       if topic.get('action') == 'BLOCKED'])

          # Print the modified output if available
          if 'outputs' in response and response['outputs']:
              print("Modified content:", response['outputs'][0].get('text', 'None'))

      return response


# Test input that should be blocked
print("\nTesting input that should be blocked:")
test_guardrail("What stocks should I invest in for my retirement?")


# Test input that should be not be blocked
print("\nTesting input that should be blocked:")
test_guardrail("How do I maintain my Sunpower X solar panel?")

##  Save Configuration for Agent Setup

Save the configuration details to be used later when setting up the Solar Panel Customer Support agent.

In [ ]:
# Create configuration dictionary
config = {
    "guardrail": {
        "id": guardrail_id,
        "version": guardrail_version,
        "name": guardrail_name,
    },
    "knowledge_base": {
        "id": kb_id,
        "name": knowledge_base_name,
    },
    "region": region
}

# Save configuration to file
config_path = "solar_panel_support_config.json"
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)
    
print(f"Configuration saved to {config_path}")

## Set up Amazon DynamoDB for Customer Profiles

In this section, we'll set up a Amazon DynamoDB table to store customer profile information. This will allow our Solar Panel Customer Support agent to retrieve and update customer information during support conversations.

In [ ]:
import sys
sys.path.append('utils')

# Now let's set up the DynamoDB table for customer profiles
# This uses the script we created in utils/customer_dynamodb.py
from customer_dynamodb import SolarCustomerDynamoDB

# Initialize the DynamoDB helper
db = SolarCustomerDynamoDB()

# Check if the table already exists both in SSM and in DynamoDB
existing_table_name = db.get_table_name_from_ssm()
table_exists = False
if existing_table_name:
    # Verify the table actually exists in DynamoDB
    table_exists = db.table_exists(existing_table_name)
    
if table_exists:
    print(f"✅ Customer profile table '{existing_table_name}' already exists.")
else:
    # Table doesn't exist yet, create it and populate with data
    # If we found a parameter but table doesn't exist, we'll create a new one
    table_name = f"SolarCustomerProfiles-{suffix}"
    print(f"Creating table '{table_name}'...")
    
    # Create the DynamoDB table
    table = db.create_table(table_name)
    
    # Generate synthetic customer profiles directly in DynamoDB
    print("Generating synthetic customer profiles...")
    customer_ids = db.generate_synthetic_profiles(count=10, table_name=table_name)
    print(f"✅ Successfully created and populated table with {len(customer_ids)} customer profiles")
    
    # Test by retrieving one profile
    sample_customer = db.get_profile_by_id(customer_ids[0], table_name)
    if sample_customer:
        print(f"Sample customer: {sample_customer['name']} from {sample_customer['country']}")

# Add the DynamoDB table name to our configuration
config["customer_table"] = existing_table_name if table_exists else table_name
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)
    
print(f"Updated configuration with customer table information")

## Summary

In this notebook, we have successfully:

1. Created a Knowledge Base for solar panel installation and maintenance information
2. Uploaded solar panel documentation to the Knowledge Base
3. Created a Guardrail to ensure safe and appropriate responses
4. Tested both the Knowledge Base and Guardrail to confirm they're working properly
5. Saved the configuration for future use when setting up the Solar Panel Customer Support agent

The next steps would be to create the Solar Panel Customer Support agent itself, which will utilize this Amazon Bedrock Knowledge Base, Guardrail and DynamoDB Database.

Now we have set up all the components needed for our Solar Panel Customer Support agent:

1. Amazon Bedrock Knowledge Base - Contains solar panel manuals and documentation
2. Amazon Bedrock Guardrail -  Safeguards and maintains appropriate responses and content filtering
3. Amazon DynamoDB Table - Stores customer profile information

These resources are ready to be used by our agent implementation.